In [1]:
# !pip install selenium requests webdriver_manager bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 36.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 46.1 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 155.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 153.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.1/178.1 kB 234.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 131.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 212.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 58.5 MB/s eta 0:00:00a 0:00:01m


In [1]:
!curl https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -o google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install --fix-broken
!google-chrome --version

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  102M  100  102M    0     0  1780k      0  0:00:58  0:00:58 --:--:-- 1582k
dpkg: error: requested operation requires superuser privilege
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Google Chrome 121.0.6167.184 


In [ ]:
!wget https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!ls

In [ ]:
! chmod +x chromedriver
!sudo mv chromedriver /usr/local/bin/
# !rm -rf /usr/local/bin/chromedriver
!ls /usr/local/bin


In [3]:
import datetime
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

def parse():
    base_url = "https://www.pararius.com/apartments/amsterdam"

    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode to avoid opening a browser window
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())

    try:
        print("Getting driver...")
        # Launch the Chrome browser with the configured options
        driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)
        

        print("Done!")
    except Exception as err:
        print("Encountered an exception!")
        raise err

    time.sleep(5)
    # Navigate to the URL
    driver.get(base_url)

    # Use explicit wait to wait for recipe elements to load
    wait = WebDriverWait(driver, 7)
    time.sleep(10)

    # Get the page source (which will contain the dynamically generated content)
    page_source = driver.page_source

    # Extract recipes from the page source
    houses = extract(page_source)
    # print(f"Houses: {houses}")

    driver.quit() # close driver

def extract(page_source) -> dict:
    # Use BeautifulSoup to parse the page source and extract the data you need
    soup = BeautifulSoup(page_source, 'html.parser')
    print(f"This is kado: \n{soup}")

if __name__ == "__main__":
    parse()


Getting driver...
Done!
This is kado: 
<html class="has-js" lang="en"><head><meta content="Az520Inasey3TAyqLyojQa8MnmCALSEU29yQFW8dePZ7xQTvSt73pHazLFTK5f7SyLUJSo2uKLesEtEa9aUYcgMAAACPeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IkRpc2FibGVUaGlyZFBhcnR5U3RvcmFnZVBhcnRpdGlvbmluZyIsImV4cGlyeSI6MTcyNTQwNzk5OSwiaXNTdWJkb21haW4iOnRydWUsImlzVGhpcmRQYXJ0eSI6dHJ1ZX0=" http-equiv="origin-trial"/>
<link href="https://cdn.cookielaw.org" rel="preconnect"/>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-V649BZ47DJ&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" crossorigin="anonymous" integrity="sha384-wEVSdqKc5hf9vkWC9kAmVRAEa11o8QNGecO6p5G2xGj+RmQhgDqFDCRdpfSr1VUf" src="https://www.gstatic.com/recaptcha/releases/moV1mTgQ6S91nuTnmll4Y9yf/recaptcha__en.js" type="text/javascript"></script><script async="" src="//resources.planetnine.com/player/new/Pnvp.js?t=0.4762061298824176&amp;h=www.pararius.com"></script><script async="" src="https://

In [ ]:
def extract(driver, page_source) -> dict:
    # Use BeautifulSoup to parse the page source and extract the data you need
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find all the links to the individual listings
    listing_links = soup.select('.listing-search-item__link')

    houses = []

    # Iterate over each link
    for link in listing_links:
        # Get the URL of the individual listing
        listing_url = link['href']

        # Navigate to the individual listing page
        driver.get(listing_url)

        # Wait for the page to load
        time.sleep(5)

        # Get the page source of the individual listing page
        listing_page_source = driver.page_source

        # Parse the individual listing page
        listing_soup = BeautifulSoup(listing_page_source, 'html.parser')

        # Extract the information you need from the individual listing page
        # For example, let's say you want to extract the description of the house
        description = listing_soup.select_one('.listing-detail-summary').text

        # Add the extracted information to the houses list
        houses.append({
            'url': listing_url,
            'description': description,
        })

    print(f"This is kado: \n{houses}")

def parse():
    # ...
    # Extract recipes from the page source
    houses = extract(driver, page_source)
    # ...


In [ ]:
import datetime
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

def parse():
    base_url = "https://www.pararius.com/apartments/amsterdam"

    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode to avoid opening a browser window
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())

    try:
        print("Getting driver...")
        # Launch the Chrome browser with the configured options
        driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)
        print("Done!")
    except Exception as err:
        print("Encountered an exception!")
        raise err

    time.sleep(5)
    # Navigate to the URL
    driver.get(base_url)

    # Use explicit wait to wait for recipe elements to load
    wait = WebDriverWait(driver, 7)
    time.sleep(10)

    # Get the page source (which will contain the dynamically generated content)
    page_source = driver.page_source

    # Extract houses from the page source
    houses = extract(driver, page_source)

    driver.quit() # close driver

def extract(driver, page_source) -> dict:
    # Use BeautifulSoup to parse the page source and extract the data you need
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find all the links to the individual listings
    listing_links = soup.select('.listing-search-item__link')

    houses = []

    # Iterate over each link
    for link in listing_links:
        # Get the URL of the individual listing
        listing_url = link['href']

        # Navigate to the individual listing page
        driver.get(listing_url)

        # Wait for the page to load
        time.sleep(5)

        # Get the page source of the individual listing page
        listing_page_source = driver.page_source

        # Parse the individual listing page
        listing_soup = BeautifulSoup(listing_page_source, 'html.parser')

        # Extract the information you need from the individual listing page
        description = listing_soup.select_one('.listing-detail-summary').text

        # Add the extracted information to the houses list
        houses.append({
            'url': listing_url,
            'description': description,
        })

    print(f"This is kado: \n{houses}")

if __name__ == "__main__":
    parse()